In [ ]:
import numpy as np 
import pandas as pd 
import pandas as pd
from ydata_profiling import ProfileReport
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
df1 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 1_EV charging reports.csv',sep=';')
df2 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 2_Hourly EV loads - Per user.csv',sep=';')
df3a = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 3a_Hourly EV loads - Aggregated private.csv',sep=';')
df3b = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 3b_Hourly EV loads - Aggregated shared.csv',sep=';')
df5 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 5_AMS data from garage Bl2.csv',sep=';')
df6 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 6_Local traffic distribution.csv',sep=';')
df_weather = pd.read_csv("/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Norway_Trondheim_ExactLoc_Weather.csv")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler


df1.dropna(inplace=True)



df1['El_kWh'] = df1['El_kWh'].str.replace(',', '').astype(float)  # Convert to numerical
df1['Duration_hours'] = df1['Duration_hours'].str.replace(',', '').astype(float)  # Convert to numerical

# One-hot encode categorical features
df1 = pd.get_dummies(df1, columns=['User_type','Garage_ID', 'User_ID', 'Shared_ID', 'month_plugin', 'weekdays_plugin', 'Plugin_category', 'Duration_category'])

In [ ]:
outlier_detector = IsolationForest(contamination=0.05)  # Adjust the contamination parameter
df1['outlier'] = outlier_detector.fit_predict(df1[['El_kWh', 'Duration_hours']])

# Remove outliers
df1 = df1[df1['outlier'] != -1]

# Feature Scaling
scaler = StandardScaler()
df1[['El_kWh', 'Duration_hours']] = scaler.fit_transform(df1[['El_kWh', 'Duration_hours']])

X = df1.drop(columns=['Duration_hours','session_ID','Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour'])  # All columns except Duration_hours
y = df1['Duration_hours']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df1 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 1_EV charging reports.csv',sep=';')
df2 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 2_Hourly EV loads - Per user.csv',sep=';')
df3a = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 3a_Hourly EV loads - Aggregated private.csv',sep=';')
df3b = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 3b_Hourly EV loads - Aggregated shared.csv',sep=';')
df5 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 5_AMS data from garage Bl2.csv',sep=';')
df6 = pd.read_csv('/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Dataset 6_Local traffic distribution.csv',sep=';')
df_weather = pd.read_csv("/kaggle/input/residential-ev-chargingfrom-apartment-buildings/archive (34)/Norway_Trondheim_ExactLoc_Weather.csv")

In [ ]:
df1['Start_plugin']

In [ ]:
df1['Start_plugin'] = pd.to_datetime(df1['Start_plugin'], format='%d.%m.%Y %H:%M')

# Extract the hour component and format it to match df6's 'Date_from'
df1['Start_plugin'] = df1['Start_plugin'].dt.strftime('%d.%m.%Y %H:00')

In [ ]:
df1.shape

In [ ]:
df1['Start_plugin']

In [ ]:
df6['Date_from'] 

In [ ]:
merged_data = df1.merge(
    df6,
    left_on='Start_plugin',
    right_on='Date_from',
)

# Drop unnecessary date and hour columns
merged_data.drop([], axis=1, inplace=True)

In [ ]:
merged_data.shape

In [ ]:
merged_data.head()

In [ ]:
merged_data['Start_plugin'] = pd.to_datetime(merged_data['Start_plugin'], format='%d.%m.%Y %H:%M')

# Extract the date component as date objects
merged_data['Start_plugin_date'] = merged_data['Start_plugin'].dt.date

In [ ]:
df_weather.datetime

In [ ]:
merged_data.Start_plugin_date

In [ ]:
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
merged_data['Start_plugin_date'] = pd.to_datetime(merged_data['Start_plugin_date'])

In [ ]:
df = merged_data.merge(
    df_weather,
    left_on='Start_plugin_date',
    right_on='datetime'
)

# Drop unnecessary date and hour columns
df.drop(['Date_from','Date_to','Unnamed: 0','datetime'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
object_columns

In [ ]:
df.to_csv("df1_traffic_weather.csv")

In [ ]:
df = df.drop(['snow_depth','revision_status','End_plugout',"Start_plugin_date"],axis=1)
df['El_kWh'] = df['El_kWh'].str.replace(',', '').astype(float)  # Convert to numerical
df['Duration_hours'] = df['Duration_hours'].str.replace(',', '').astype(float)  # Convert to numerical

# One-hot encode categorical features
df = pd.get_dummies(df, columns=['User_type','KROPPAN BRU', 'MOHOLTLIA', 'SELSBAKK','Garage_ID', 'User_ID', 'Shared_ID', 'month_plugin', 'weekdays_plugin', 'Plugin_category', 'Duration_category'])

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
scaler = StandardScaler()
df[['El_kWh', 'Duration_hours']] = scaler.fit_transform(df[['El_kWh', 'Duration_hours']])

X = df.drop(columns=['Duration_hours','session_ID','Start_plugin', 'Start_plugin_hour', 'End_plugout_hour'])  # All columns except Duration_hours
y = df['Duration_hours']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

In [ ]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE)
    :param y_true: Array of true values
    :param y_pred: Array of predicted values
    :return: SMAPE score
    """
    return 100 * (2 * abs(y_pred - y_true) / (abs(y_true) + abs(y_pred))).mean()

# RANDOM FOREST REGRESSOR

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
print("Random Forest Regressor:")
print("Score:", rf.score(X_test, y_test))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))
print("Root Mean Squared Error:", sqrt(mean_squared_error(y_test, y_pred)))
print("Symmetric Mean Absolute Percentage Error:", symmetric_mean_absolute_percentage_error(y_test, y_pred))

# LGBM REGRESSOR

In [ ]:
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)

In [ ]:
print("LGBM Regressor:")
print("Score:", lgbm.score(X_test, y_test))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))
print("Root Mean Squared Error:", sqrt(mean_squared_error(y_test, y_pred)))
print("Symmetric Mean Absolute Percentage Error:", symmetric_mean_absolute_percentage_error(y_test, y_pred))

# XGB Regressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

In [ ]:
print("XGB Regressor:")
print("Score:", xgb.score(X_test, y_test))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))
print("Root Mean Squared Error:", sqrt(mean_squared_error(y_test, y_pred)))
print("Symmetric Mean Absolute Percentage Error:", symmetric_mean_absolute_percentage_error(y_test, y_pred))

# GRADIENT BOOSTING REGRESSOR

In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)

In [ ]:
print("Gradient Boosting Regressor:")
print("Score:", gbr.score(X_test, y_test))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))
print("Root Mean Squared Error:", sqrt(mean_squared_error(y_test, y_pred)))
print("Symmetric Mean Absolute Percentage Error:", symmetric_mean_absolute_percentage_error(y_test, y_pred))

# GRAPHS

In [ ]:
# MSE

import matplotlib.pyplot as plt

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i] + 0.003, y[i], ha = 'center')
        

mse = [0.6176, 0.6074, 0.5991, 0.5761]
names = ['Random Forest', 'LGBM', 'XGB', 'Gradient Boosting']
plt.bar(names, mse, color='red')
addlabels(names, mse)
plt.ylim(0.5, 0.65)
plt.plot(mse, marker='o', color='black')
plt.title('Mean Squared Error')
plt.xlabel('Regressors')
plt.show()


In [ ]:
# MAE

import matplotlib.pyplot as plt

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i] + 0.003, y[i], ha = 'center')
        

mse = [0.4199, 0.4526, 0.4453, 0.4488]
names = ['Random Forest', 'LGBM', 'XGB', 'Gradient Boosting']
plt.bar(names, mse, color='green')
addlabels(names, mse)
plt.ylim(0.4, 0.475)
plt.plot(mse, marker='o', color='red')
plt.title('Mean Absolute Error')
plt.xlabel('Regressors')
plt.show()

In [ ]:
y_pred_gbr = gbr.predict(X_test)
residuals = y_test - y_pred_gbr
for_x = [x for x in range(len(residuals))]

plt.scatter(for_x, residuals, marker='o', color='blue', linestyle='-')
plt.title('Error Prediction Plot - Gradient Boosting')
plt.xlabel('Data Points')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True)
plt.show()

In [ ]:
y_pred_rf = rf.predict(X_test)
residuals = y_test - y_pred_rf
for_x = [x for x in range(len(residuals))]

plt.scatter(for_x, residuals, marker='o', color='blue', linestyle='-')
plt.title('Error Prediction Plot - Random Forest')
plt.xlabel('Data Points')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True)
plt.show()

In [ ]:
y_pred_lgbm = lgbm.predict(X_test)
residuals = y_test - y_pred_lgbm
for_x = [x for x in range(len(residuals))]

plt.scatter(for_x, residuals, marker='o', color='blue', linestyle='-')
plt.title('Error Prediction Plot - LGBM')
plt.xlabel('Data Points')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True)
plt.show()

In [ ]:
y_pred_xgb = xgb.predict(X_test)
residuals = y_test - y_pred_xgb
for_x = [x for x in range(len(residuals))]

plt.scatter(for_x, residuals, marker='o', color='blue', linestyle='-')
plt.title('Error Prediction Plot - XGB')
plt.xlabel('Data Points')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True)
plt.show()